In [ ]:
def read_fasta(file_path):
    # Readfile function to return the 2 sequences in a fasta file
    sequences = []  # Initialize an empty list to store sequences
    with open(file_path, 'r') as file:
        sequence = ''  # Initialize string
        for line in file:
            if line.startswith('>'):
                # If the line starts with '>', it's a header line (can use to store sequence info?)
                if sequence != '':
                    sequences.append(sequence)
                    sequence = ''  # Reset sequence for the next one
            else:
                # Remove any leading/trailing whitespace and convert to uppercase
                sequence += line.strip().upper()
        sequences.append(sequence)
    return sequences


def slippage_aware_alignment(seq1, seq2, match_score, mismatch_score, cs, cn):
    #   seq1, seq2: The two sequences to be aligned
    #   match_score: Score for a nucleotide match
    #   mismatch_score: Score for a nucleotide mismatch
    #   cs: Slippage gap penalty (when the gap character matches the previous character)
    #   cn: Non-slippage gap penalty (standard gap penalty)

    m = len(seq1)
    n = len(seq2)

    # Initialize the scoring matrix and the traceback matrix
    score = [[0] * (n + 1) for _ in range(m + 1)]  # (m+1) x (n+1) matrix for scores
    traceback = [[None] * (n + 1) for _ in range(m + 1)]  # Matrix to store traceback directions

    # Initialize the first row of the scoring matrix
    for j in range(1, n + 1):
        if j > 1 and seq2[j - 1] == seq2[j - 2]:
            gap_penalty = cs
        else:
            gap_penalty = cn
        score[0][j] = score[0][j - 1] + gap_penalty
        traceback[0][j] = 'left'  # Indicate the move came from the left

    # Initialize the first column of the scoring matrix
    for i in range(1, m + 1):
        if i > 1 and seq1[i - 1] == seq1[i - 2]:
            gap_penalty = cs
        else:
            gap_penalty = cn
        score[i][0] = score[i - 1][0] + gap_penalty
        traceback[i][0] = 'up'  # Indicate the move came from above

    # Fill the rest of the scoring matrix
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            # Calculate diagonal score (match or mismatch)
            if seq1[i - 1] == seq2[j - 1]:
                diag_score = score[i - 1][j - 1] + match_score  # Match
            else:
                diag_score = score[i - 1][j - 1] + mismatch_score  # Mismatch

            # Calculate up score (insertion in seq2/deletion in seq1)
            if i > 1 and seq1[i - 1] == seq1[i - 2]: # Check for slippage in seq1
                gap_penalty_S = cs
            else:
                gap_penalty_S = cn
            up_score = score[i - 1][j] + gap_penalty_S  # Score from moving up

            # Calculate left score (insertion in seq1/deletion in seq2)
            if j > 1 and seq2[j - 1] == seq2[j - 2]: # Check the slippage for seq2
                gap_penalty_T = cs
            else:
                gap_penalty_T = cn
            left_score = score[i][j - 1] + gap_penalty_T  # Score from moving left

            # Choose the maximum score among diagonal, up, and left moves
            max_score = max(diag_score, up_score, left_score)
            score[i][j] = max_score  # Update the scoring matrix with the maximum score

            # Record the direction of the move in the traceback matrix
            if max_score == diag_score:
                traceback[i][j] = 'diag'
            elif max_score == up_score:
                traceback[i][j] = 'up'
            else:
                traceback[i][j] = 'left'

    # Traceback to reconstruct the optimal alignment
    aligned_seq1 = ''
    aligned_seq2 = ''
    i, j = m, n

    while i > 0 or j > 0:
        if traceback[i][j] == 'diag':
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            i -= 1
            j -= 1
        elif traceback[i][j] == 'up':
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = '_' + aligned_seq2 # Add gap in seq2
            i -= 1
        elif traceback[i][j] == 'left':
            aligned_seq1 = '_' + aligned_seq1 # Add gap in seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            j -= 1
        else:
            break  # Reached the start of the alignment

    return score[m][n], aligned_seq1, aligned_seq2  # Return the optimal score (at m,n) and the aligned sequences


def main():
    # Algorithm inputs
    #fasta_file = '/content/drive/MyDrive/hw1_brca1_3utr_small.fa.txt'  # File 1
    fasta_file = '/content/drive/MyDrive/hw1_brca1_3utr_full.fa.txt'  # File 2
    match_score = int(1)
    mismatch_score = int(-1)
    cs = int(-1)
    cn = int(-2)

    # Read the sequences from the FASTA file and assign each sequence to a variable
    sequences = read_fasta(fasta_file)
    seq1 = sequences[0]
    seq2 = sequences[1]

    # Perform the slippage-aware alignment
    score, aligned_seq1, aligned_seq2 = slippage_aware_alignment(seq1, seq2, match_score, mismatch_score, cs, cn)

    # Print the optimal alignment score and the aligned sequences
    print("Optimal Alignment Score:", score)
    print("Alignment:")
    print(aligned_seq1)
    print(aligned_seq2)


if __name__ == "__main__":
    main()

Optimal Alignment Score: 215
Alignment:
TTCAGG_CTGTTGTTGGCTTAGGGCTGGAAGCAC_AGAGTGGCT_TG__GCCTC_A_AG_AGAATAGCTG_GT_TTCCCTAAGTTTACTTCTCTAAAACCCTGTGTTC_ACAAAGGCAGAGAGTCAGACCCTTCAA__T___GGAAGGAGAGTGCTTGGGATCGATTATGTGACTTAAAGTCAGAATAGT___C_CTTGGGC_AG_T_TCTC__A__AAT_GTTGGA_G_TGGAACAT_TGG_GGAGGAAATTCT_G_AGGCAGGTATTAGAAATGAAAAGG_AAACTTGAAACCTGGGCATGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCA_AGGTGGGCAGATCACTGGAG__GT_CAGGAGTTCGAA_ACCAGCCTGGCCAACATGGTGAAACCCCATCTCTACTAAAAATACAGAAATTAGCCGGTCATGGTGGTGGACACCTGTAATCCCAGCTACTCAGGTGGCTAAGGCAGGAGAATCACTTCAGCCCGGGAGGT_GGAGGTTGCAGTGAGCCAAGATCATACCACGGCACTCCAGCCTGGGTGACAGTGAGACTGTGGCTCAAAAAAAAAAAAAAAAAAAGGAAAATGAAACTAGAAGAGATTTCTAAAAGTCTGAGATATATTTGCTAGATTTCTAAAGAATGTGTTCTAAAACAGCAGAAGATTTTCAAGAACCGGTTTCCAAAGACAGTCTTCTAATTCCTCATTAG_TAATAAGTAAAATGTTTAT_TGTTGTAGC_TCTGGTATATAATCCATTCCTCTTAAAATATAAG_ACCTCTGGCATGAATATTTCATATCTATAAAATGACAGATCCCACCAGGA_AGGAAGCTGTTGCTTTCTTTGAGGTGATTTTTTTCCT_TTGCTCCCTGTTGCTGAAACCATACAGCTTCATAAATAATTTTGCTTGCTGAAGGAAGAAAAAGTGTTTTTCATAAAC